# phase 1 - data preparation

In [1]:
import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maciej\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Maciej\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## original data exploration

In [ ]:
# select first 100 000 records

chunks = pd.read_json("../data/yelp_academic_dataset_review.json", lines=True, chunksize=100000)

for chunk in chunks:
    chunk.to_json("../data/yelp_academic_dataset_review_short.json", orient="records", lines=True)
    break

In [6]:
# check first records, columns and missing information

data = pd.read_json("../data/yelp_academic_dataset_review_short.json", lines=True)

print("HEAD")
print(data.head())
print("\nCOLUMNS")
print(data.columns)
print("\nINFO")
print(data.info())


HEAD
                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm... 2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delic

## data cleaning

In [7]:
# drop irrelevant columns, process text and check for missing values

def clean_text(text: str) -> str:
    # remove special characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)

    # tokenize text
    tokens = word_tokenize(text)

    # remove stop words
    stop_words = set(stopwords.words("english"))
    stop_words.discard('not')
    filtered_tokens = [token for token in tokens if token.casefold() not in stop_words]

    # lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # convert tokens to lowercase
    lowercase_tokens = [token.lower() for token in lemmatized_tokens]

    # join tokens into a single string
    return " ".join(lowercase_tokens)

# drop irrelevant columns
irrelevant_columns = ["review_id", "user_id", "business_id", "useful", "funny", "cool", "date"]
for col in irrelevant_columns:
    if col in data.columns:
        print(f"Dropping: {col}")
        data = data.drop(columns=[col])

# apply cleaning function to data
data["text"] = data["text"].apply(clean_text)

# save cleaned data to a file
with open("../data/yelp_reviews_cleaned.csv", mode="a", newline="", encoding="utf-8") as file:
    data.to_csv(file, index=False)

# view cleaned data columns
print("COLUMNS")
print(data.columns)

Dropping: review_id
Dropping: user_id
Dropping: business_id
Dropping: useful
Dropping: funny
Dropping: cool
Dropping: date
COLUMNS
Index(['stars', 'text'], dtype='object')


## split data into training and testing sets

In [9]:
df = pd.read_csv("../data/yelp_reviews_cleaned.csv")

# split into 80/20

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["stars"], test_size=0.2, random_state=42
)

train_texts.to_csv("../data/80_20/train_texts.csv")
train_labels.to_csv("../data/80_20/train_labels.csv")
test_texts.to_csv("../data/80_20/test_texts.csv")
test_labels.to_csv("../data/80_20/test_labels.csv")


# split into 70/30

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["stars"], test_size=0.3, random_state=42
)

train_texts.to_csv("../data/70_30/train_texts.csv")
train_labels.to_csv("../data/70_30/train_labels.csv")
test_texts.to_csv("../data/70_30/test_texts.csv")
test_labels.to_csv("../data/70_30/test_labels.csv")